In [4]:
#python program to automate the process of scrapping data from instagram by logging into your account
#importing required libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import time
import os
import wget
import json
import pandas as pd


def search_for_keyword(username,password,keyword):  
    global driver
    #initializing chrome driver for controling Chrome from program 
    driver = webdriver.Chrome(r"C:/Users/vivek/Desktop/chromedriver_win32/chromedriver.exe")
    #maximizing window
    driver.maximize_window()
    #opening instagram in the Chrome
    driver.get('https://www.instagram.com/')
    #navigating to username and password fields in the Instagram login page
    username_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
    password_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))
    #clearing the username and password fields
    username_field.clear()
    password_field.clear()
    #passing username and password
    username_field.send_keys(username)
    password_field.send_keys(password)
    #clicking on the log in button
    try :
        log_in =  WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
    except:
        pass
    time.sleep(10)
    check = driver.find_element(By.CSS_SELECTOR,'body')
    check = check.get_attribute('class')
    if check == '_a3wf _-kb segoe':
        return "ERROR-1"
    #cliclking on the Not Now buttons if they appear on the screen 
    try:
        not_now =  WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH , '//button[contains(text(),"Not Now")]'))).click()
        not_now_2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH , '//button[contains(text(),"Not Now")]'))).click()
    except:
        pass
    try:
        search = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div[class='_aacl _aacp _aacu _aacx _aada']"))).click()
        search_box = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH , '//input[@placeholder="Search"]')))
    except:
        search_box = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH , '//input[@placeholder="Search"]')))
    #to clear the search box
    search_box.clear()
    #to pass the keyword to the search box
    search_box.send_keys(keyword)
    time.sleep(5)
    #to click on the first result that appear after searching a keyword 
    account_click = driver.find_element(By.CSS_SELECTOR,'div[class="_aacl _aaco _aacw _aacx _aad6"]')
    keyword_new = account_click.text
    account_click.click()
    #to check whether the given keyword is a hashtag or a username
    if keyword_new[0] == '#':
        #to scrap using the hashtag 
        scrap_using_hashtag(keyword_new)
    else:
        #to scrap using the keyword
        scrap_using_username(keyword_new)
      
    
def scrap_using_hashtag(keyword_new):
    time.sleep(15)
    #to find the number of posts for a given hashtag
    hashtag_posts = driver.find_element(By.CSS_SELECTOR, 'div[class="_aacl _aacp _aacu _aacx _aad6 _aade"]').text
    #for setting the path in the computer to store the output data
    path = setting_path(keyword_new[1:])
    old_path = path
    os.mkdir(path)
    path = os.path.join(path, keyword_new + ' Hashtag Images')
    os.mkdir(path)
    #scrolling the webpage using the javascript method
    driver.execute_script('window.scrollTo(0,10000);')
    #finding the elements in the webpage with the img tag name
    images = driver.find_elements(By.CSS_SELECTOR, 'img')
    images = [image.get_attribute('src') for image in images]
    counter = 0
    print("Downloading Images ...")
    for image in images:
        save_as = os.path.join(path, keyword_new + " Image " + str(counter+1) + ".jpg")
        #to download the images that are selected 
        wget.download(image, save_as)
        counter += 1
    print("\nData is saved at ",old_path)
    
def scrap_using_username(keyword_new):
    time.sleep(10)
    #to get the user profile details from the page
    user_info = driver.find_elements(By.CSS_SELECTOR, "div[class='_aacl _aacp _aacu _aacx _aad6 _aade']")
    user_info_new = [user.text for user in user_info]
    user_posts = user_info_new[0]
    user_followers = user_info_new[1]
    user_following = user_info_new[2]
    user_bio = user_info_new[3]
    user_username = keyword_new
    try:
        external_links = driver.find_element(By.CSS_SELECTOR, "div[class='_aacl _aacp _aacw _aacz _aada _aade']").text
    except:
        external_links = "None"
    path = setting_path(keyword_new)
    old_path = path
    os.mkdir(path)
    #storing the data in a dictionary
    dictionary = {
        "Username": user_username,
        "Posts": user_posts,
        "Followers": user_followers,
        "Follwing":user_following,
        "Bio":user_bio,
        "External links in profile":external_links
    }
 
    # Serializing json
    json_object = json.dumps(dictionary, indent=6)
 
    # Writing to sample.json
    file_name = path + "\\" + keyword_new + " account info.json"
    with open(file_name, "w") as outfile:
        outfile.write(json_object)
        
    #clicking on followers 
    followers = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH , "//div[contains(text(),'followers')]"))).click()
    time.sleep(5)
    driver.execute_script("window.scrollTo(0,4000);")
    #getting the usernames of the followers
    followers_usernames = driver.find_elements(By.CSS_SELECTOR, "div[class=' _ab8y  _ab94 _ab97 _ab9f _ab9k _ab9p _abcm']")
    time.sleep(30)
    followers_usernames = [follower_username.text for follower_username in followers_usernames]
    d1 = {"Usernames":followers_usernames}
    #converting into a dataframe
    df = pd.DataFrame(d1)
    df.index += 1
    #data is stored in a csv file
    csv_file = df.to_csv()
    file_name_followers = path + "\\" + keyword_new + " Followers Usernames.csv"
    with open(file_name_followers,'w') as follow_file:
        follow_file.write(csv_file)
    #clicking on the cancel button that appear on the folowers page
    cancel = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[class="_ab8w  _ab94 _ab99 _ab9f _ab9m _ab9p  _ab9y _abcm"]'))).click()
    
    time.sleep(5)
    #clicking on following
    following = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH , "//div[contains(text(),'following')]"))).click()
    time.sleep(5)
    driver.execute_script("window.scrollTo(0,10000);")
    #to extract the usernames of the following members
    following_usernames = driver.find_elements(By.CSS_SELECTOR, "div[class=' _ab8y  _ab94 _ab97 _ab9f _ab9k _ab9p _abcm']") 
    time.sleep(20)
    following_usernames = [following_username.text for following_username in following_usernames]
    d1 = {"Usernames":following_usernames}
    #converting into dataframe
    df = pd.DataFrame(d1)
    df.index += 1
    csv_file = df.to_csv()
    file_name_following = path + "\\" + keyword_new + " Following Usernames.csv"
    with open(file_name_following,'w') as following_file:
        following_file.write(csv_file)
    #clicking on the cancel button on the following page
    cancel = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[class="_ab8w  _ab94 _ab99 _ab9f _ab9m _ab9p  _ab9y _abcm"]'))).click()
    
    time.sleep(5)
    #setting path to store the images
    path = os.path.join(path, keyword_new + ' account Images')
    os.mkdir(path)
    driver.execute_script('window.scrollTo(0,10000);')
    images = driver.find_elements(By.CSS_SELECTOR, 'img')
    images = [image.get_attribute('src') for image in images]
    length = len(images)
    counter = 0
    print("Downloading Images ...")
    for image in images:
        if counter != (length-1):
            save_as = os.path.join(path, keyword_new + " Image " + str(counter+1) + ".jpg")
            #to download the images using their the url
            wget.download(image, save_as)
        counter += 1
    print("\nData is saved at ",old_path)
        
def setting_path(keyword_new):
    path = 'C:\\'
    path = os.path.join(path, keyword_new + " Data")
    return path

def log_in_credentials():
    print("\nLog In Credentials for Instagram")
    #for taking username as a input from the user
    username = input("Enter Username : ")
    #password from the user
    password = input("Enter Password : ")
    #keyword to search for
    keyword = input("Enter the Username or Hashtag to search in Instagram : ")
    result = search_for_keyword(username,password,keyword)
    check_status(result)

def check_status(result):
    #if the credentials entered by the user are incorrect
    if result == "ERROR-1":
        print("Failed To Login To Your Instagram Account !!!,Invalid input credentials or poor internet connection")
        try_again_status = input("DO YOU WANT TO TRY AGAIN(Y or N) :")
        #want to try again or not
        if try_again_status.upper() == 'Y': 
            log_in_credentials()
        else:
            print("\nProgram Terminated !!!")
            pass
    else:
        try_again_status = input("DO YOU WANT TO TRY AGAIN(Y or N) :")
        #want to try again or not
        if try_again_status.upper() == 'Y': 
            log_in_credentials()
        else:
            print("\nProgram Terminated !!!")
            pass
log_in_credentials()
#code by vivek


Log In Credentials for Instagram
Enter Username : unique__username___
Enter Password : unique@password
Enter the Username or Hashtag to search in Instagram : #ipl2023


C:\Users\vivek\AppData\Local\Temp\ipykernel_11908\1794156258.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:/Users/vivek/Desktop/chromedriver_win32/chromedriver.exe")


100% [................................................................................] 5225 / 5225
Data is saved at  C:\ipl2023 Data
DO YOU WANT TO TRY AGAIN(Y or N) :y

Log In Credentials for Instagram
Enter Username : jbdiuf
Enter Password : dfiggisdugfigsd
Enter the Username or Hashtag to search in Instagram : udfiu
Failed To Login To Your Instagram Account !!!,Invalid input credentials or poor internet connection
DO YOU WANT TO TRY AGAIN(Y or N) :N

Program Terminated !!!
